# Cluster Characters

In [ ]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein

In [16]:
import spacy
from spacy import displacy
import en_core_web_sm

In [3]:
from collections import Counter

In [4]:
import pandas as pd
pd.set_option("max_rows", 400)

In [6]:
import networkx 
from networkx.algorithms.components.connected import connected_components, node_connected_component

In [7]:
import itertools
from fuzzywuzzy import fuzz

In [10]:
#nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

In [12]:
filepath = "../texts/literature/Jones-Lost-in-The-City.txt"

In [14]:
text = open(filepath).read()
document = nlp(text)

# Get People (More Accurately)

Here's a common spaCy NER scenario:

You process your text with spaCy, and you find that the model has correctly tagged a person as a "PERSON." Nice! 🎉

But then, paragraphs later, you notice that spaCy has tagged the exact same person as a different entity — perhaps an organization ("ORG") or a place ("GPE"). Ugh 😫 

To get a more accurate character/people count, we're going to extract all the named entities that spaCy identified as a "PERSON" and then count *any* instance of that entitiy, regardless of its NER label.

Additionally, we're going to output this character list to a CSV file, so we can clean and edit the list by hand (if we wish).

Extract list of all named entities labeled "PERSON":

In [20]:
spacy_identified_people = []

for named_entity in document.ents:
    if named_entity.label_ == "PERSON":
        
        spacy_identified_people.append(named_entity.text)

Output list of identified people to a CSV file for manual cleaning and editing:

In [21]:
pd.DataFrame(Counter(spacy_identified_people).most_common(), columns=['character', 'count']).to_csv('spacy-identified-people.csv', index=False)

After manual editing, re-upload CSV file for accurate list of people:

In [22]:
spacy_identified_people = pd.read_csv('spacy-identified-people.csv')['character'].tolist()

Count any entity that matches a person in our cleaned list of people. Also extract the [index number](https://spacy.io/usage/linguistic-features#named-entities-101) where the person appears in the document.

In [23]:
all_people_matches = []
all_people_matches_plus_ids = []

#Get all entity matches for a previously identified person
for named_entity in document.ents:
    if named_entity.text in spacy_identified_people:
        person = named_entity.text
        
        #Remove apostrophe 's from character name
        person = person.replace("’s", "").strip()
        #Get the character index number from the text
        person_index = named_entity.start_char
        
        all_people_matches.append(person)
        all_people_matches_plus_ids.append([person, person_index])

In [110]:
people_tally = Counter(all_people_matches)
character_df = pd.DataFrame(people_tally.most_common())
character_df.columns = ['character', 'count']

character_df

character  count
0                 Cassandra    113
1                     Anita     90
2                     Joyce     90
3                    Caesar     74
4                 Madeleine     69
5                    Vivian     67
6                   Melanie     65
7                    Rickey     62
8                   Sherman     60
9                    Maddie     58
10                  Woodrow     53
11                 Santiago     50
12                    Penny     49
13                    Marie     45
14                Betsy Ann     44
15                    Manny     44
16                    Jenny     43
17                   Rhonda     40
18                    Diane     40
19                   Marcus     38
20                   Samuel     38
21                  Garrett     37
22                      Sam     33
23                 Humphrey     33
24                   Robert     32
25                    Maude     31
26                    Pearl     29
27                    Carol     29
28                 Beatrice     29
29                     Avis     27
30                  Mildred     26
31                    Ralph     25
32                 Vernelle     24
33                    Miles     21
34                   Gladys     21
35                   Pookie     20
36                     Rita     19
37                    Clara     18
38                Anacostia     18
39                 Marvella     18
40                  Carmena     17
41                 Wilamena     17
42                  Jenkins     16
43                   Marvin     16
44                    Sandy     16
45                   Lonney     15
46                     Anna     14
47                    Jesus     14
48                   Arnisa     13
49                   Wesley     12
50                   Smokey     12
51                   Thelma     11
52                     Adam     11
53                  Darlene     10
54              Gospelteers     10
55                 Saunders     10
56               John Henry     10
57                   Baxter      9
58                   Lorton      9
59                    Hazel      9
60                    Alice      8
61                   Angelo      8
62                   Brenda      8
63                 Patricia      8
64                       Vi      8
65                   Sandra      7
66                    Tommy      7
67                   Clovis      7
68                    kinda      7
69                Jesse Mae      7
70                   Elaine      7
71                     Etta      6
72                 Ballston      6
73                  Clarine      6
74                   Easter      6
75                   Taylor      6
76                   sittin      6
77                      Joe      6
78                  Calhoun      6
79                   Walter      5
80                   Dwayne      5
81                   Turner      5
82                       Al      5
83                  Antibes      5
84               Jessie Mae      5
85               John Smith      5
86            Vernelle Wise      5
87            Robert Morgan      4
88           Mildred Harper      4
89                    Honey      4
90                    Roger      4
91                     Yall      4
92                  Frazier      4
93            Chuck Taylors      4
94                   Sawyer      4
95                    Smith      4
96                   George      4
97                Nicodemus      4
98                    Janet      3
99                   Carlos      3
100                  Nothin      3
101                 Jeffrey      3
102             Joyce Moses      3
103                     Joy      3
104                     Pat      3
105             Aunt Maddie      3
106      Martin Luther King      3
107                Williams      3
108                   Walsh      3
109                Cornelia      3
110              Mama Joyce      3
111                  Morton      3
112                    John      3
113   Woodrow L. Cunningham 

# Cluster Characters By Name Similarity and Distance


spaCy doesn't know that "Betsy Ann Morgan" and "Betsy Ann" should be the same person. So we're also going to pair two character names if they're an extremely close match and they occur within 750 characters of one another.

In [111]:
aggregated_people = []

threshold_distance = 750

#Get all entity matches for a previously identified person
for person, another_person in itertools.combinations(all_people_matches_plus_ids, 2):
        distance = abs(person[1] - another_person[1])
        if distance < threshold_distance:
            
            if person[0] != another_person[0]:
                
                if fuzz.partial_ratio(person[0], another_person[0]) == 100:
                    aggregated_people.append((person[0], another_person[0]))

0:00:01.053097


In [112]:
G=networkx.Graph()
G.add_edges_from(aggregated_people)
people_clusters  = list(connected_components(G))
people_clusters = [sorted(cluster, key=len, reverse=True) for cluster in people_clusters]
people_clusters

[['Betsy Ann Morgan', 'Betsy Ann'],
 ['Miles Patterson', 'Miles'],
 ['Walter Creed', 'Creed'],
 ['Jenny Creed', 'Jenny'],
 ['Robert Morgan', 'Robert'],
 ['Janet Gordon', 'Janet'],
 ['Ralph Holley', 'Ralph'],
 ['LaDeidre Gordon', 'LaDeidre'],
 ['Thelma Holley', 'Thelma'],
 ['Darlene Greenley', 'Darlene'],
 ['Etta O’Connell', 'Etta'],
 ['Mary Keith', 'Mary'],
 ['Blondelle Harris', 'Blondelle'],
 ['Cassandra G. Lewis', 'Cassandra'],
 ['Rhonda Ferguson', 'Rhonda'],
 ['Melanie Cartwright', 'Melanie'],
 ['Anita Hughes', 'Anita'],
 ['Gladys Harper', 'Gladys', 'Glady'],
 ['Joyce Moses', 'Mama Joyce', 'Joyce'],
 ['Pearl Guthrie', 'Pearl Malone', 'Pearl'],
 ['Manny Soto', 'Manny'],
 ['Sherman Wheeler', 'Sherman'],
 ['Sandra Wallington', 'Sandra'],
 ['Angelo Billings', 'Angelo'],
 ['Burberry Carol', 'Carol'],
 ['Lonney McCrae', 'Lonney'],
 ['Brenda Roper', 'Brenda'],
 ['Mabel Smith', 'Mabel'],
 ['Penny Jenkins', 'Jenkins'],
 ['Al Jenkins', 'Al'],
 ['Joy Lambert', 'Joy'],
 ['handcuff Penny', 'Penn

In [113]:
def add_clustered_characters(row):
    character = row
    if any(character in cluster for cluster in people_clusters):
        for cluster in people_clusters:
            if character in cluster:
                return " // ".join(cluster)
    else:
        return character

In [115]:
character_df['clustered_characters'] = character_df['character'].apply(add_clustered_characters)

In [116]:
character_df

character  count  \
0                 Cassandra    113   
1                     Anita     90   
2                     Joyce     90   
3                    Caesar     74   
4                 Madeleine     69   
5                    Vivian     67   
6                   Melanie     65   
7                    Rickey     62   
8                   Sherman     60   
9                    Maddie     58   
10                  Woodrow     53   
11                 Santiago     50   
12                    Penny     49   
13                    Marie     45   
14                Betsy Ann     44   
15                    Manny     44   
16                    Jenny     43   
17                   Rhonda     40   
18                    Diane     40   
19                   Marcus     38   
20                   Samuel     38   
21                  Garrett     37   
22                      Sam     33   
23                 Humphrey     33   
24                   Robert     32   
25                    Maude     31   
26                    Pearl     29   
27                    Carol     29   
28                 Beatrice     29   
29                     Avis     27   
30                  Mildred     26   
31                    Ralph     25   
32                 Vernelle     24   
33                    Miles     21   
34                   Gladys     21   
35                   Pookie     20   
36                     Rita     19   
37                    Clara     18   
38                Anacostia     18   
39                 Marvella     18   
40                  Carmena     17   
41                 Wilamena     17   
42                  Jenkins     16   
43                   Marvin     16   
44                    Sandy     16   
45                   Lonney     15   
46                     Anna     14   
47                    Jesus     14   
48                   Arnisa     13   
49                   Wesley     12   
50                   Smokey     12   
51                   Thelma     11   
52                     Adam     11   
53                  Darlene     10   
54              Gospelteers     10   
55                 Saunders     10   
56               John Henry     10   
57                   Baxter      9   
58                   Lorton      9   
59                    Hazel      9   
60                    Alice      8   
61                   Angelo      8   
62                   Brenda      8   
63                 Patricia      8   
64                       Vi      8   
65                   Sandra      7   
66                    Tommy      7   
67                   Clovis      7   
68                    kinda      7   
69                Jesse Mae      7   
70                   Elaine      7   
71                     Etta      6   
72                 Ballston      6   
73                  Clarine      6   
74                   Easter      6   
75                   Taylor      6   
76                   sittin      6   
77                      Joe      6   
78                  Calhoun      6   
79                   Walter      5   
80                   Dwayne      5   
81                   Turner      5   
82                       Al      5   
83                  Antibes      5   
84               Jessie Mae      5   
85               John Smith      5   
86            Vernelle Wise      5   
87            Robert Morgan      4   
88           Mildred Harper      4   
89                    Honey      4   
90                    Roger      4   
91                     Yall      4   
92                  Frazier      4   
93            Chuck Taylors      4   
94                   Sawyer      4   
95                    Smith      4   
96                   George      4   
97                Nicodemus      4   
98                    Janet      3   
99                   Carlos      3   
100                  Nothin      3   
101                 Jeffrey      3   
102             Joyce Moses      3   
103                     Joy      3   
104                     Pat 

Manually edit

In [38]:
#character_df.to_csv('clustered_characters_draft_Lost.csv')

In [39]:
#character_df = pd.read_csv('clustered_characters_edited.csv')

In [117]:
character_df.groupby('clustered_characters')[['count']].sum().sort_values(by='count', ascending=False).reset_index()

clustered_characters  count
0                      Cassandra G. Lewis // Cassandra    114
1                   Joyce Moses // Mama Joyce // Joyce     96
2                                Anita Hughes // Anita     92
3    Marie Delaveaux Wilson // Marie heard Vernelle...     77
4                                         Vivian // Vi     75
5    Samuel Lamont Williams // Samuel Williams // S...     75
6                                               Caesar     74
7                                            Madeleine     69
8                        Melanie Cartwright // Melanie     66
9                             Rickey Madison // Rickey     64
10                          Sherman Wheeler // Sherman     62
11            Maddie Williams // Aunt Maddie // Maddie     62
12                                             Woodrow     53
13                                            Santiago     50
14                             handcuff Penny // Penny     50
15                                 Manny Soto // Manny     45
16                       Betsy Ann Morgan // Betsy Ann     45
17                                Jenny Creed // Jenny     44
18                           Diane McCollough // Diane     42
19                           Rhonda Ferguson // Rhonda     42
20                              Ida Garrett // Garrett     38
21                                              Marcus     38
22                             Robert Morgan // Robert     36
23                  Beatrice Atwell // Beatrice // Bea     33
24                                            Humphrey     33
25                             Maude Townsend // Maude     32
26              Pearl Guthrie // Pearl Malone // Pearl     31
27                             Burberry Carol // Carol     30
28                           Mildred Harper // Mildred     30
29                                                Avis     27
30                               Ralph Holley // Ralph     26
31                            Miles Patterson // Miles     23
32                    Gladys Harper // Gladys // Glady     23
33                  Rita Hadley // Sister Rita // Rita     22
34                           Pookie Williams // Pookie     21
35                            Carmena Boone // Carmena     20
36                          Wilamena Mason // Wilamena     18
37                                           Anacostia     18
38                                               Clara     18
39                                            Marvella     18
40                            Penny Jenkins // Jenkins     17
41                                               Sandy     16
42                             Lonney McCrae // Lonney     16
43                                              Marvin     16
44                             Arnisa Isaacs // Arnisa     14
45                                                Anna     14
46                                               Jesus     14
47                            Smokey Peebles // Smokey     13
48                                              Wesley     12
49               Snowball Patterson // Patricia // Pat     12
50                             Thelma Holley // Thelma     12
51           Elaine Cunningham // Cunningham // Elaine     11
52                         Darlene Greenley // Darlene     11
53                                                Adam     11
54                                          John Henry     10
55                                         Gospelteers     10
56                                            Saunders     10
57                               Lady Baxter // Baxter     10
58                              Brenda Roper // Brenda      9
59                           Angelo Billings // Angelo      9
60                                 John Smith // Smith      9
61                                               Hazel      9
62                       Jesse Mae Carson // Jesse Mae      9
63                                              Lorton      9
64  

# Make a Network of Characters

In [118]:
edge_list = []

threshold_distance = 50

#Get all entity matches for a previously identified person
for person, another_person in itertools.combinations(all_people_matches_plus_ids, 2):
        distance = abs(person[1] - another_person[1])
        if distance < threshold_distance:
            
            if person[0] != another_person[0]:
                
                edge_list.append((person[0], another_person[0]))

In [119]:
character_df = pd.DataFrame(Counter(edge_list).most_common(), columns=['character_pair', 'edge_weight'])
character_df['character1']=character_df['character_pair'].str[0]
character_df['character2']=character_df['character_pair'].str[1]

In [120]:
character_network = character_df[['character1', 'character2', 'edge_weight']]
character_network[:100]

character1    character2  edge_weight
0       Anita       Melanie           12
1   Cassandra         Anita           11
2   Cassandra       Melanie           10
3     Melanie     Cassandra           10
4       Anita     Cassandra            9
5      Marcus          Avis            9
6      Caesar       Sherman            8
7         Sam     Madeleine            8
8    Santiago        Rickey            8
9       Jenny        Robert            7
10    Melanie         Anita            7
11   Santiago         Joyce            7
12      Jenny         Clara            6
13      Joyce         Pearl            6
14      Carol          Anna            6
15     Arnisa           Sam            6
16      Joyce      Humphrey            6
17      Joyce        Rickey            6
18     Vivian         Diane            6
19   Beatrice       Garrett            6
20   Wilamena         Marie            6
21      Clara        Robert            5
22     Rhonda     Cassandra            5
23     Caesar         Manny            5
24      Manny        Caesar            5
25     Lonney        Brenda            5
26     Samuel     Madeleine            5
27  Madeleine        Samuel            5
28     Rickey      Santiago            5
29   Santiago      Humphrey            5
30     Rickey         Joyce            5
31    Woodrow          Rita            5
32    Garrett      Beatrice            5
33  Cassandra        Rhonda            4
34      Anita        Gladys            4
35      Pearl         Joyce            4
36    Melanie        Gladys            4
37      Manny       Sherman            4
38    Sherman        Caesar            4
39     Caesar         Carol            4
40      Penny      Patricia            4
41  Madeleine           Sam            4
42      Diane        Vivian            4
43      Maude         Anita            4
44      Maude        Vivian            4
45      Maude         Diane            4
46     Vivian         Maude            4
47  Betsy Ann         Jenny            3
48     Robert         Clara            3
49       Etta     Betsy Ann            3
50  Anacostia     Cassandra            3
51     Gladys     Cassandra            3
52     Gladys       Melanie            3
53    Sherman        Sandra            3
54     Angelo        Caesar            3
55      Manny         Carol            3
56         Al         Penny            3
57       Avis        Marcus            3
58     Lorton     Madeleine            3
59  Madeleine        Pookie            3
60     Maddie        Samuel            3
61  Madeleine        Maddie            3
62        Sam         Hazel            3
63  Madeleine        Arnisa            3
64   Humphrey         Joyce            3
65      Joyce         Sandy            3
66      Sandy         Joyce            3
67      Anita        Vivian            3
68     Vivian         Anita            3
69      Marie      Wilamena            3
70      Jenny  Walter Creed            2
71     Robert         Jenny            2
72     Carlos      Carleton            2
73    Darlene         Ralph            2
74  Betsy Ann       Darlene            2
75    Darlene     Betsy Ann            2
76       Mary     Blondelle            2
77     Gladys         Anita            2
78     Dwayne       Melanie            2
79      Anita        Rhonda            2
80    Melanie        Wesley            2
81     hearin       Melanie            2
82    Jeffrey        Rhonda            2
83    Sherman         Manny            2
84     Sandra       Sherman            2
85     Angelo       Sherman            2
86    Sherman        Angelo            2
87     Caesar        Angelo            2
88       Anna        Caesar            2
89     Caesar          Anna            2
90       Anna         Carol            2
91     Baxter         Penny            2
92   Marvella        Marvin            2
93     Samuel        Maddie            2
94     Pookie        Maddie            2
95     Maddie        Pookie            2
96  Madeleine   Aunt Maddi

In [122]:
def add_clustered_characters(row):
    character = row
    if any(character in cluster for cluster in people_clusters):
        for cluster in people_clusters:
            if character in cluster:
                return " // ".join(cluster)
    else:
        return character

In [123]:
character_network['character1'] = character_network['character1'].apply(add_clustered_characters)

/Users/melaniewalsh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [124]:
character_network['character2'] = character_network['character2'].apply(add_clustered_characters)

/Users/melaniewalsh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [130]:
character_network.to_csv('lost-in-the-city-network.csv', index=False, encoding='utf-8')

In [128]:
character_network.groupby(['character1', 'character2'])[['edge_weight']].sum().sort_values(by='edge_weight', ascending=False).reset_index()

character1  \
0    Samuel Lamont Williams // Samuel Williams // S...   
1                                Anita Hughes // Anita   
2                      Cassandra G. Lewis // Cassandra   
3                      Cassandra G. Lewis // Cassandra   
4                        Melanie Cartwright // Melanie   
5                                Anita Hughes // Anita   
6                                            Madeleine   
7                                               Marcus   
8                              Arnisa Isaacs // Arnisa   
9                   Beatrice Atwell // Beatrice // Bea   
10                                            Santiago   
11                                              Caesar   
12                       Melanie Cartwright // Melanie   
13                                        Vivian // Vi   
14                  Joyce Moses // Mama Joyce // Joyce   
15                                Jenny Creed // Jenny   
16   Marie Delaveaux Wilson // Marie heard Vernelle...   
17                                            Santiago   
18                             Burberry Carol // Carol   
19                          Wilamena Mason // Wilamena   
20                  Joyce Moses // Mama Joyce // Joyce   
21                  Joyce Moses // Mama Joyce // Joyce   
22                                Jenny Creed // Jenny   
23                                              Caesar   
24                                 Manny Soto // Manny   
25                                             Woodrow   
26                          Sherman Wheeler // Sherman   
27                             Maude Townsend // Maude   
28                                 Manny Soto // Manny   
29                                              Caesar   
30                            Rickey Madison // Rickey   
31                                        Vivian // Vi   
32                                           Madeleine   
33                                            Santiago   
34                           Rhonda Ferguson // Rhonda   
35            Maddie Williams // Aunt Maddie // Maddie   
36                              Ida Garrett // Garrett   
37                             Lonney McCrae // Lonney   
38                                               Clara   
39                            Rickey Madison // Rickey   
40                             Maude Townsend // Maude   
41                     Cassandra G. Lewis // Cassandra   
42              Pearl Guthrie // Pearl Malone // Pearl   
43                          Calhoun Lambeth // Calhoun   
44                           Diane McCollough // Diane   
45   Samuel Lamont Williams // Samuel Williams // S...   
46                          Sherman Wheeler // Sherman   
47                               Anita Hughes // Anita   
48                               Anita Hughes // Anita   
49                             Maude Townsend // Maude   
50                       Melanie Cartwright // Melanie   
51                           Angelo Billings // Angelo   
52                             handcuff Penny // Penny   
53                                        Vivian // Vi   
54   Samuel Lamont Williams // Samuel Williams // S...   
55                                Jenny Creed // Jenny   
56                  Joyce Moses // Mama Joyce // Joyce   
57                    Gladys Harper // Gladys // Glady   
58                              Ida Garrett // Garrett   
59                                    Al Jenkins // Al   
60                    Gladys Harper // Gladys // Glady   
61                                  Mary Keith // Mary   
62                                           Anacostia   
63   Marie Delaveaux Wilson // Marie heard Vernelle...   
64                         Darlene Greenley // Darlene   
65                              Etta O’Connell // Etta   
66                                              Lorton   
67                                               Sandy   
68                                        

In [21]:
filepath = "../texts/literature/Little-Women.txt"

In [22]:
text = open(filepath).read()

In [4]:
chunked_text= text.split('\n')

In [7]:
chunked_text[2]

'                           PLAYING PILGRIMS.'

In [40]:
number_of_chunks = 5000
chunked_text = [text[i:i+number_of_chunks] for i in range(0, len(text), number_of_chunks)]

In [41]:
len(chunked_text)

216

In [140]:
chunked_documents = list(nlp.pipe(chunked_text))

# Get People (More Accurately)

Extract list of all named entities labeled "PERSON":

In [142]:
spacy_identified_people = []

for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "PERSON":
            spacy_identified_people.append(named_entity.text)

Output list of identified people to a CSV file for manual cleaning and editing:

In [21]:
#pd.DataFrame(Counter(spacy_identified_people).most_common(), columns=['character', 'count']).to_csv('spacy-identified-people.csv', index=False)

After manual editing, re-upload CSV file for accurate list of people:

In [22]:
#spacy_identified_people = pd.read_csv('spacy-identified-people.csv')['character'].tolist()

In [205]:
all_people_matches = []
all_people_matches_plus_ids = []
document_length = 0

#Get all entity matches for a previously identified person
for document in chunked_documents:
    document_length += len(document.text)
    for named_entity in document.ents:
        if named_entity.text in spacy_identified_people:
            person = named_entity.text

            #Remove apostrophe 's from character name
            person = person.replace("’s", "").strip()
            #Get the character index number from the text
            
            person_index =  (document_length - named_entity.start_char)

            all_people_matches.append(person)
            all_people_matches_plus_ids.append([person, person_index])

In [207]:
people_tally = Counter(all_people_matches)
character_df = pd.DataFrame(people_tally.most_common())
character_df.columns = ['character', 'count']

character_df

character  count
0                        Jo   1296
1                       Amy    631
2                    Laurie    582
3                       Meg    523
4                      Beth    463
..                      ...    ...
541                 Michael      1
542            Gutenberg-tm      1
543  Project Gutenberg-tm's      1
544        Gregory B. Newby      1
545         Michael S. Hart      1

[546 rows x 2 columns]

# Make a network!

In [219]:
edge_list = []

threshold_distance = 100

#Get all entity matches for a previously identified person
for person, another_person in itertools.combinations(all_people_matches_plus_ids, 2):
        distance = abs(person[1] - another_person[1])
        if distance < threshold_distance:
            
            if person[0] != another_person[0]:
                
                edge_list.append((person[0], another_person[0]))

In [220]:
character_df = pd.DataFrame(Counter(edge_list).most_common(), columns=['character_pair', 'edge_weight'])
character_df['character1']=character_df['character_pair'].str[0]
character_df['character2']=character_df['character_pair'].str[1]

In [222]:
character_network = character_df[['character1', 'character2', 'edge_weight']]
character_network[character_network['edge_weight'] > 2]

character1   character2  edge_weight
0            Meg           Jo           98
1         Laurie           Jo           97
2             Jo       Laurie           94
3             Jo          Amy           92
4             Jo         Beth           89
5            Amy           Jo           85
6           Beth           Jo           83
7             Jo          Meg           79
8           Beth          Amy           60
9            Amy       Laurie           52
10        Laurie          Amy           45
11         March           Jo           37
12           Amy         Beth           35
13           Amy          Meg           32
14           Meg          Amy           32
15           Meg         Beth           31
16        Laurie          Meg           30
17          Beth          Meg           27
18           Meg       Laurie           24
19        Brooke          Meg           24
20             _           Jo           23
21      Laurence           Jo           22
22          John          Meg           22
23        Hannah           Jo           21
24           Meg         John           21
25      Laurence       Laurie           19
26         Bhaer           Jo           18
27          Beth       Hannah           17
28        Laurie         Beth           17
29         Teddy           Jo           17
30            Jo            _           16
31            Jo        March           15
32          Beth       Laurie           14
33         March          Amy           13
34             _          Meg           13
35           Meg       Brooke           13
36        Hannah         Beth           12
37         March          Meg           12
38            Jo        Bhaer           12
39    Aunt March           Jo           11
40         March         Beth           11
41           Meg       Hannah           11
42            Jo       Hannah           11
43           Meg            _           11
44           Amy            _           11
45        Laurie            _           11
46        Brooke       Laurie           10
47             _          Amy           10
48             _       Laurie            9
49    Aunt March          Amy            9
50          John           Jo            9
51           Amy        March            8
52            Jo         Fred            8
53          Fred           Jo            8
54          Beth            _            8
55             _         Beth            8
56         March       Laurie            8
57            Jo        Teddy            8
58          Beth        March            7
59             _       Hannah            7
60            Jo     Laurence            7
61        Hannah       Laurie            7
62        Hannah          Meg            7
63           Amy       Hannah            7
64           Amy        Davis            7
65        Sallie          Meg            7
66          Kate           Jo            7
67            Jo       Brooke            7
68        Brooke           Jo            7
69        Hannah          Amy            7
70           Meg        March            6
71            Jo   Aunt March            6
72      Laurence         Beth            6
73           Meg       Sallie            6
74        Laurie        March            6
75        Laurie     Laurence            6
76         Daisy         Demi            6
77        Sallie           Jo            5
78           Amy     Laurence            5
79         Belle          Meg            5
80         Teddy          Amy            5
81      Laurence       Hannah            5
82            Jo         John            5
83        Brooke         John            5
84      Margaret           Jo            4
85      Roderigo         Zara            4
86          Beth     Laurence            4
87        Moffat          Meg            4
88           Ned       Laurie            4
89       Crocker           Jo            4
90        Brooke        March            4
91           Meg         Kate            4
92     

In [223]:
character_network[character_network['edge_weight'] > 2].to_csv('Little-Women-character-network.csv')